# Ensemble

- 일련의 예측기 (즉, 분류나 회귀 모델)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있다. 일련의 예측기를 ensemble이라고 부르기 때문에 이를 Ensemble learning이라고 한다. 앙상블 학습 알고리즘을 Ensemble method라고 한다.
- 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 일련의 결정 트리 분류기를 훈련시킬 수 있다. 예측을 하려면 모든 개별 트리의 예측을 구하면 된다. 그런 다음 가장 많은 선택을 많은(major vote) 클래스를 예측 결과값으로 한다. 결정 트리의 대표적인 ensemble을 Random Forest라고 한다. 간단한 방법이지만 Random Forest는 오늘날 가장 강력한 머신러닝 알고리즘 중 하나이며, 현업에서도 많이 사용된다.
- 대표적인 ensemble 기법
 - Bagging
 - Boosting
 - Stacking

# 1. Voting based classifier (투표 기반 분류기)
 - 더 좋은 분류기를 만드는 방법은 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것
 - Hard voting classifier : 다수결 투표로 정해지는 분류기
  - 각 개별 분류기(base classifier) 중 가장 뛰어난 것보다도 정확도가 높은 경우가 존재
  - Base classifer가 week learner(약한 학습기, 랜덤 추측보다 조금 더 좋은 성능을 내는 분류기)일지라도, 충분하게 많고 다양하다면 strong learner(강한 학습기, 높은 정확도를 내는 분류기)가 될 수 있음
  - Law of large numbers(큰 수의 법칙)
    - 동전 던지기 예제 : 앞면이 나올 확률이 51%이고 뒷면이 나올 확률이 49%인 동전이 존재한다고 하자. 동전을 1,000번 던졌을 때, 앞면이 다수가 나올 확률은 얼마일까? 이항분포를 사용하면 성공 확률이 51%인 동전을 1,000번 던져서 앞면이 1번만 나올 확률은 $\frac{1000}{1} \times 0.51^{1} \times(1-0.51)^{1000-1}=1.6\times 10^{-307}$이다. 이런식으로 499까지의 확률을 더하고 1에서 빼면 된다.
    - 51% 정확도를 가진 1,000개의 분류기로 앙상블 모델을 구축한다면 75%의 정확도를 기대할 수 있다. (분류기가 완벽하게 독립 & 오차의 상관관계가 없어야 가능)
  - Ensemble 방법은 예측기가 가능한 한 서로 독립적일 때 최고의 성능을 발휘. 다양한 분류기를 얻는 한 가지 방법은 **각기 다른 알고리즘으로 학습**시킨다. 이렇게 하면 매우 다른 종류의 오차를 만들 가능성이 높기 때문에 앙상블 모델의 정확도를 향상

In [4]:
# 동전 던지기 예제
from scipy import stats

print(1 - stats.binom.cdf(499, 1000, 0.51))  # 천 번 던졌을 때 앞면이 절반 이상 나올 확률
print(1 - stats.binom.cdf(4999, 10000, 0.51))  # 만 번 던졌을 때 앞면이 절반 이상 나올 확률

0.7467502275561786
0.9777976478701533


Skleran에서는 투표 기반 분류기를 `VotingClassifier`로 제공하고, base learner로 서로 다른 모델들을 사용할 수 있도록 해준다.

In [119]:
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split

x, y = datasets.make_moons(n_samples=1000, noise=0.3, random_state=42)  # moons data 사용
train_x, test_x, train_y, test_y = train_test_split(x, y, random_state=42)

> Hard voting : Base learner의 결과값이 클래스이며, major class를 사용하여 최종 결과를 도출한다.

In [120]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Train
log_clf = LogisticRegression(solver="liblinear", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf = SVC(gamma="auto", random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', logistic), ('rf', rf), ('svm', svm)], voting='hard'
)

In [121]:
# 개별 학습기의 성능
for clf in (rf, logistic, svm, voting_clf):
    clf.fit(train_x, train_y)
    test_pred = clf.predict(test_x)
    print(clf.__class__.__name__, accuracy_score(test_y, test_pred))

RandomForestClassifier 0.896
LogisticRegression 0.852
SVC 0.912
VotingClassifier 0.908


> Soft voting : Base learner의 결과값이 확률이며, 각 learner의 확률의 평균을 내어 최종 결과를 도출한다.

In [122]:
log_clf = LogisticRegression(solver="liblinear", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf = SVC(gamma="auto", random_state=42, probability=True)

voting_clf = VotingClassifier(
    estimators=[('lr', logistic), ('rf', rf), ('svm', svm)], voting='soft'
)

In [123]:
# 개별 학습기의 성능
for clf in (rf, logistic, svm, voting_clf):
    clf.fit(train_x, train_y)
    test_pred = clf.predict(test_x)
    print(clf.__class__.__name__, accuracy_score(test_y, test_pred))

RandomForestClassifier 0.912
LogisticRegression 0.852
SVC 0.912
VotingClassifier 0.92
